In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Build Vertex AI Experiment lineage for custom training

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/build_model_experimentation_lineage_with_prebuild_code.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/build_model_experimentation_lineage_with_prebuild_code.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/experiments/build_model_experimentation_lineage_with_prebuild_code.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Overview

As a Data Scientist, you want to be able to reuse code path (data preprocessing, feature engineering etc...) that others within your team have written to simplify and standardize all the complex data wrangling. 

### Objective

In this notebook, you learn how to integrate preprocessing code in a Vertex AI experiments. Also you build the experiment lineage lets you record, analyze, debug, and audit metadata and artifacts produced along your ML journey.

### Dataset

This dataset is the UCI News Aggregator Data Set which contains 422,937 news collected between March 10th, 2014 and August 10th, 2014. Below are example records from the dataset:


|ID |TITLE                                                                |URL                                                                                                                        |PUBLISHER        |CATEGORY|STORY                        |HOSTNAME           |TIMESTAMP    |
|---|---------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------|-----------------|--------|-----------------------------|-------------------|-------------|
|1  |Fed official says weak data caused by weather, should not slow taper |http://www.latimes.com/business/money/la-fi-mo-federal-reserve-plosser-stimulus-economy-20140310,0,1312750.story\?track=rss|Los Angeles Times|b       |ddUyU0VZz0BRneMioxUPQVP6sIxvM|www.latimes.com    |1394470370698|
|2  |Fed's Charles Plosser sees high bar for change in pace of tapering   |http://www.livemint.com/Politics/H2EvwJSK2VE6OF7iK1g3PP/Feds-Charles-Plosser-sees-high-bar-for-change-in-pace-of-ta.html   |Livemint         |b       |ddUyU0VZz0BRneMioxUPQVP6sIxvM|www.livemint.com   |1394470371207|
|3  |US open: Stocks fall after Fed official hints at accelerated tapering|http://www.ifamagazine.com/news/us-open-stocks-fall-after-fed-official-hints-at-accelerated-tapering-294436                |IFA Magazine     |b       |ddUyU0VZz0BRneMioxUPQVP6sIxvM|www.ifamagazine.com|1394470371550|
|4  |Fed risks falling 'behind the curve', Charles Plosser says           |http://www.ifamagazine.com/news/fed-risks-falling-behind-the-curve-charles-plosser-says-294430                             |IFA Magazine     |b       |ddUyU0VZz0BRneMioxUPQVP6sIxvM|www.ifamagazine.com|1394470371793|
|5  |Fed's Plosser: Nasty Weather Has Curbed Job Growth                   |http://www.moneynews.com/Economy/federal-reserve-charles-plosser-weather-job-growth/2014/03/10/id/557011                   |Moneynews        |b       |ddUyU0VZz0BRneMioxUPQVP6sIxvM|www.moneynews.com  |1394470372027|

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or Vertex AI Workbench Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip3 install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

### Install additional packages

Install additional package dependencies not installed in your notebook environment,TensorFlow or Vertex AI SDK. Use the latest major GA version of each package.

In [1]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install {USER_FLAG} --upgrade joblib fsspec gcsfs scikit-learn -q
! pip install {USER_FLAG} --upgrade google-cloud-aiplatform -q

     |████████████████████████████████| 2.3 MB 6.9 MB/s 
     |████████████████████████████████| 47 kB 3.9 MB/s 
     |████████████████████████████████| 106 kB 73.0 MB/s 
     |████████████████████████████████| 233 kB 65.5 MB/s 
     |████████████████████████████████| 115 kB 64.5 MB/s 
     |████████████████████████████████| 408 kB 61.4 MB/s 
     |████████████████████████████████| 106 kB 56.2 MB/s 
     |████████████████████████████████| 107 kB 61.9 MB/s 
     |████████████████████████████████| 107 kB 62.4 MB/s 
     |████████████████████████████████| 107 kB 24.6 MB/s 
     |████████████████████████████████| 106 kB 61.4 MB/s 
     |████████████████████████████████| 106 kB 64.0 MB/s 
     |████████████████████████████████| 106 kB 66.9 MB/s 
     |████████████████████████████████| 106 kB 28.6 MB/s 
     |████████████████████████████████| 105 kB 46.9 MB/s 
     |████████████████████████████████| 105 kB 57.7 MB/s 
     |████████████████████████████████| 105 kB 32.8 MB/s 
     |███████████

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [2]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable APIs](https://console.cloud.google.com/flows/enableapi?apiid=cloudresourcemanager.googleapis.com,aiplatform.googleapis.com).

1. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [23]:
PROJECT_ID = "sv-ml-sandbox"  # @param {type:"string"}

In [24]:
import os

if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [25]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


Otherwise, set your project ID here.

In [8]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "sv-ml-sandbox"  # @param {type:"string"}

In [9]:
!gcloud config set project $PROJECT_ID

Updated property [core/project].


#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [26]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

#### UUID

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a uuid for each instance session, and append it onto the name of resources you create in this tutorial.

In [11]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### Authenticate your Google Cloud account

**If you are using Vertex AI Workbench Notebooks**, your environment is already
authenticated.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type and select
the following role into the filter box:

    *   Storage Admin
    *   Storage Object Admin
    *   Service Account User
    *   Vertex AI Administrator


5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [31]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code

IS_COLAB = False

if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:

        IS_COLAB = True

        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.

    elif not os.getenv("IS_TESTING"):

        %env GOOGLE_APPLICATION_CREDENTIALS ''

In [32]:
SERVICE_ACCOUNT = "412996116194-compute@developer.gserviceaccount.com"  # @param {type:"string"}

In [33]:

import sys

IS_COLAB = "google.colab" in sys.modules

if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

In [34]:
! print($SERVICE_ACCOUNT)

/bin/bash: -c: line 0: syntax error near unexpected token `412996116194-compute@developer.gserviceaccount.com'
/bin/bash: -c: line 0: ` print(412996116194-compute@developer.gserviceaccount.com)'


### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [20]:
BUCKET_NAME = "sv-ml-sandbox-bucket"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [21]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = f"gs://{BUCKET_NAME}"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [35]:
! gsutil ls -al $BUCKET_URI

      4136  2022-05-11T22:42:01Z  gs://sv-ml-sandbox-bucket/aiplatform-2022-05-11-22:42:00.923-aiplatform_custom_trainer_script-0.1.tar.gz#1652308921166372  metageneration=1
      4134  2022-05-11T22:55:42Z  gs://sv-ml-sandbox-bucket/aiplatform-2022-05-11-22:55:42.460-aiplatform_custom_trainer_script-0.1.tar.gz#1652309742592080  metageneration=1
      4135  2022-05-11T23:06:45Z  gs://sv-ml-sandbox-bucket/aiplatform-2022-05-11-23:06:45.707-aiplatform_custom_trainer_script-0.1.tar.gz#1652310405786633  metageneration=1
       297  2022-05-11T23:06:08Z  gs://sv-ml-sandbox-bucket/mean_and_std.json#1652310368518297  metageneration=1
                                 gs://sv-ml-sandbox-bucket/data/
                                 gs://sv-ml-sandbox-bucket/iris/
                                 gs://sv-ml-sandbox-bucket/model/
                                 gs://sv-ml-sandbox-bucket/pipeline_root/
                                 gs://sv-ml-sandbox-bucket/pipelines/
                         

In [30]:
! gsutil ls

ServiceException: 401 Anonymous caller does not have storage.buckets.list access to the Google Cloud project.


In [36]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

No changes made to gs://sv-ml-sandbox-bucket/
No changes made to gs://sv-ml-sandbox-bucket/


### Set project folder

In [37]:
DATA_PATH = "data"
!mkdir -m 777 -p {DATA_PATH}

### Get the data

In [38]:
DATASET_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip"
!wget --no-parent {DATASET_URL} --directory-prefix={DATA_PATH}
!mkdir -m 777 -p {DATA_PATH}/temp {DATA_PATH}/raw
!unzip {DATA_PATH}/*.zip -d {DATA_PATH}/temp
!mv {DATA_PATH}/temp/*.csv {DATA_PATH}/raw && rm -Rf {DATA_PATH}/temp && rm -f {DATA_PATH}/*.zip

--2022-10-12 12:57:52--  https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29224203 (28M) [application/x-httpd-php]
Saving to: ‘data/NewsAggregatorDataset.zip’

NewsAggregatorDatas 100%[===================>]  27.87M  58.8MB/s    in 0.5s    

2022-10-12 12:57:52 (58.8 MB/s) - ‘data/NewsAggregatorDataset.zip’ saved [29224203/29224203]

Archive:  data/NewsAggregatorDataset.zip
  inflating: data/temp/2pageSessions.csv  
   creating: data/temp/__MACOSX/
  inflating: data/temp/__MACOSX/._2pageSessions.csv  
  inflating: data/temp/newsCorpora.csv  
  inflating: data/temp/__MACOSX/._newsCorpora.csv  
  inflating: data/temp/readme.txt    
  inflating: data/temp/__MACOSX/._readme.txt  


In [39]:
!gsutil -m cp -R {DATA_PATH}/raw $BUCKET_URI/{DATA_PATH}/raw

Copying file://data/raw/2pageSessions.csv [Content-Type=text/csv]...
Copying file://data/raw/newsCorpora.csv [Content-Type=text/csv]...
/ [2/2 files][100.5 MiB/100.5 MiB] 100% Done                                    
Operation completed over 2 objects/100.5 MiB.                                    


### Import libraries

In [40]:
# General
import logging

logger = logging.getLogger("logger")
logging.basicConfig(level=logging.INFO)

import collections
import tempfile
import time
from json import dumps

collections.Iterable = collections.abc.Iterable

# Vertex AI
from google.cloud import aiplatform as vertex_ai

### Define constants

In [41]:
# Base
DATASET_NAME = "news_corpora"
DATASET_URI = f"{BUCKET_URI}/{DATA_PATH}/raw/newsCorpora.csv"

# Experiments
TASK = "classification"
MODEL_TYPE = "naivebayes"
EXPERIMENT_NAME = f"{TASK}-{MODEL_TYPE}-{UUID}"
EXPERIMENT_RUN_NAME = "run-1"

# Preprocessing
PREPROCESSED_DATASET_NAME = f"preprocessed_{DATASET_NAME}"
PREPROCESS_EXECUTION_NAME = "preprocess"
COLUMN_NAMES = [
    "id",
    "title",
    "url",
    "publisher",
    "category",
    "story",
    "hostname",
    "timestamp",
]
DELIMITER = "	"
INDEX_COL = 0
PREPROCESSED_DATASET_URI = (
    f"{BUCKET_URI}/{DATA_PATH}/preprocess/{PREPROCESSED_DATASET_NAME}.csv"
)

# Training
TRAIN_EXECUTION_NAME = "train"
TARGET = "category"
FEATURES = "title"
TEST_SIZE = 0.2
SEED = 8
TRAINED_MODEL_URI = f"{BUCKET_URI}/deliverables/{UUID}"
MODEL_NAME = f"{EXPERIMENT_NAME}-model"

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [42]:
vertex_ai.init(
    project=PROJECT_ID, experiment=EXPERIMENT_NAME, staging_bucket=BUCKET_URI
)

### Set pre-built containers

Set the pre-built Docker container image for training and prediction.


For the latest list, see [Pre-built containers for training](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers).


For the latest list, see [Pre-built containers for prediction](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers).

In [43]:
SERVE_IMAGE = vertex_ai.helpers.get_prebuilt_prediction_container_uri(
    framework="sklearn", framework_version="1.0", accelerator="cpu"
)

### Initialize Experiment run

In [44]:
run = vertex_ai.start_run(EXPERIMENT_RUN_NAME)

Associating projects/412996116194/locations/us-central1/metadataStores/default/contexts/classification-naivebayes-ah8czu22-run-1 to Experiment: classification-naivebayes-ah8czu22


INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/412996116194/locations/us-central1/metadataStores/default/contexts/classification-naivebayes-ah8czu22-run-1 to Experiment: classification-naivebayes-ah8czu22


## Model experimentation with pre-built data pre-processing code

### Data preprocessing

In this step, you simulate some preprocessing on raw data to create the training dataset.

Indeed it would happen that you have some data preprocessing that others within your team may have written. For that reason, you need a way to integrate the preprocessing code in your experiment runs to standardize and reuse all the complex data wrangling that you are dealing with. 

Using Vertex AI Experiments, you can track that code as part of the run execution by adding one `with` statement in your experiment context. 

#### Create a Dataset Metadata Artifact

First you create the Dataset artifact to track the dataset resource in the Vertex ML Metadata and create the experiment lineage.

In [45]:
raw_dataset_artifact = vertex_ai.Artifact.create(
    schema_title="system.Dataset", display_name=DATASET_NAME, uri=DATASET_URI
)

#### Create a preprocess module

Next, you build a simple preprocessing module to convert the text case and remove the punctuation.

In [46]:
"""
Preprocess module
"""


import pandas as pd


def preprocess(df: pd.DataFrame, text_col: str) -> pd.DataFrame:
    """
    Preprocess text
    Args:
        df: The DataFrame to preprocesss
        text_col: The text column name
    Returns:
        preprocessed_df: The datafrane with text in lowercase and without punctuation
    """
    preprocessed_df = df.copy()
    preprocessed_df[text_col] = preprocessed_df[text_col].apply(lambda x: x.lower())
    preprocessed_df[text_col] = preprocessed_df[text_col].apply(
        lambda x: x.translate(str.maketrans("", "", string.punctuation))
    )
    return preprocessed_df

#### Add the `preprocessing` Execution

Vertex AI Experiments supports tracking both executions and artifacts. Executions are steps in an ML workflow that can include but are not limited to data preprocessing, training, and model evaluation. Executions can consume artifacts such as datasets and produce artifacts such as models.

You add the preprocessing step to track its execution in the lineage associated to Vertex AI Experiment. 
For Vertex AI, the parameters are passed inside the message field which we see in the logs. These structures of the logs are predefined.

In [47]:
with vertex_ai.start_execution(
    schema_title="system.ContainerExecution", display_name=PREPROCESS_EXECUTION_NAME
) as exc:
    logging.info(f"Start {PREPROCESS_EXECUTION_NAME} execution.")
    exc.assign_input_artifacts([raw_dataset_artifact])

    # Log preprocessing params --------------------------------------------------
    logging.info("Log preprocessing params.")
    vertex_ai.log_params(
        {
            "delimiter": DELIMITER,
            "features": dumps(COLUMN_NAMES),
            "index_col": INDEX_COL,
        }
    )

    # Preprocessing ------------------------------------------------------------
    logging.info("Preprocessing.")
    raw_df = pd.read_csv(
        raw_dataset_artifact.uri,
        delimiter=DELIMITER,
        names=COLUMN_NAMES,
        index_col=INDEX_COL,
    )
    preprocessed_df = preprocess(raw_df, "title")
    preprocessed_df.to_csv(PREPROCESSED_DATASET_URI, sep=",")

    # Log preprocessing metrics and store dataset artifact ---------------------
    logging.info(f"Log preprocessing metrics and {PREPROCESSED_DATASET_NAME} dataset.")
    vertex_ai.log_metrics(
        {
            "n_records": preprocessed_df.shape[0],
            "n_columns": preprocessed_df.shape[1],
        },
    )

    preprocessed_dataset_metadata = vertex_ai.Artifact.create(
        schema_title="system.Dataset",
        display_name=PREPROCESSED_DATASET_NAME,
        uri=PREPROCESSED_DATASET_URI,
    )
    exc.assign_output_artifacts([preprocessed_dataset_metadata])

### Model Training

In this step, you train a multinomial Naive Bayes pipeline.

#### Create model training module

Below the training module.

**get_training_split :** It takes parameters like  x(The data to be split), y(The labels to be split), test_size(The proportion of the data to be reserved for testing) and random_state(The seed used by the random number generator).
This function return training data, testing data , The training labels and The testing labels.

**get_pipeline :** It return's the model.

**train_pipeline :** It train the model by using model, training data, training lables and return's the trained model.

**evaluate_model :** It evaluate the model and return the accuracy of the model.


In [48]:
"""
Train module
"""

import joblib
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


def get_training_split(
    x: pd.DataFrame, y: pd.Series, test_size: float, random_state: int
) -> (pd.DataFrame, pd.Series, pd.DataFrame, pd.Series):
    """
    Splits data into training and testing sets
    Args:
        x: The data to be split
        y: The labels to be split
        test_size: The proportion of the data to be reserved for testing
        random_state: The seed used by the random number generator
    Returns:
        x_train: The training data
        x_test: The testing data
        y_train: The training labels
        y_test: The testing labels
    """

    x_train, x_val, y_train, y_val = train_test_split(
        x, y, test_size=test_size, random_state=random_state
    )
    return x_train, x_val, y_train, y_val


def get_pipeline():
    """
    Get the model
    Args:
        None
    Returns:
        model: The model
    """
    model = Pipeline(
        [
            ("vect", CountVectorizer()),
            ("tfidf", TfidfTransformer()),
            ("clf", MultinomialNB()),
        ]
    )
    return model


def train_pipeline(model: Pipeline, X_train: pd.Series, y_train: pd.Series) -> Pipeline:
    """
    Train the model
    Args:
        model: The model to train
        X_train: The training data
        y_train: The training labels
    Returns:
        model: The trained model
    """
    model.fit(X_train, y_train)
    return model


def evaluate_model(model: Pipeline, X_test: pd.Series, y_test: pd.Series) -> float:
    """
    Evaluate the model
    Args:
        model: The model to evaluate
        X_test: The testing data
        y_test: The testing labels
    Returns:
        score: The accuracy of the model
    """
    # Evaluate model
    y_pred = model.predict(X_test)

    # Store evaluation metrics
    # Store evaluation metrics
    metrics = {
        "accuracy": round(accuracy_score(y_test, y_pred), 5),
        "precision": round(precision_score(y_test, y_pred, average="weighted"), 5),
        "recall": round(recall_score(y_test, y_pred, average="weighted"), 5),
    }
    return metrics


def save_model(model: Pipeline, save_path: str) -> int:
    try:
        with tempfile.NamedTemporaryFile() as tmp:
            joblib.dump(trained_pipeline, filename=tmp.name)
            ! gsutil cp {tmp.name} {save_path}/model.joblib
    except RuntimeError as error:
        print(error)
    return 1

#### Add the `train` Execution

Next, you add the training task to the experiment execution to update the experiment lineage.

In [49]:
with vertex_ai.start_execution(
    schema_title="system.ContainerExecution", display_name=TRAIN_EXECUTION_NAME
) as exc:

    exc.assign_input_artifacts([preprocessed_dataset_metadata])

    # Get training and testing data
    logging.info("Get training and testing data.")
    x_train, x_val, y_train, y_val = get_training_split(
        preprocessed_df[FEATURES],
        preprocessed_df[TARGET],
        test_size=TEST_SIZE,
        random_state=SEED,
    )
    # Get model pipeline
    logging.info("Get model pipeline.")
    pipeline = get_pipeline()

    # Log training param -------------------------------------------------------

    # Log data parameters
    logging.info("Log data parameters.")
    vertex_ai.log_params(
        {
            "target": TARGET,
            "features": FEATURES,
            "test_size": TEST_SIZE,
            "random_state": SEED,
        }
    )

    # Log pipeline parameters
    logging.info("Log pipeline parameters.")
    vertex_ai.log_params(
        {
            "pipeline_steps": dumps(
                {step[0]: str(step[1].__class__.__name__) for step in pipeline.steps}
            )
        }
    )

    # Training -----------------------------------------------------------------

    # Train model pipeline
    logging.info("Train model pipeline.")
    train_start = time.time()
    trained_pipeline = train_pipeline(pipeline, x_train, y_train)
    train_end = time.time()

    # Evaluate model
    logging.info("Evaluate model.")
    model_metrics = evaluate_model(trained_pipeline, x_val, y_val)

    # Log training metrics and store model artifact ----------------------------

    # Log training metrics
    logging.info("Log training metrics.")
    vertex_ai.log_metrics(model_metrics)

    # Generate first ten predictions
    logging.info("Generate prediction sample.")
    prediction_sample = trained_pipeline.predict(x_val)[:10]
    print("prediction sample:", prediction_sample)

    # Upload Model on Vertex AI
    logging.info("Upload Model on Vertex AI.")
    loaded = save_model(trained_pipeline, TRAINED_MODEL_URI)
    if loaded:
        model = vertex_ai.Model.upload(
            serving_container_image_uri=SERVE_IMAGE,
            artifact_uri=TRAINED_MODEL_URI,
            display_name=MODEL_NAME,
        )

    exc.assign_output_artifacts([model])

prediction sample: ['m' 'e' 'e' 't' 'e' 'b' 'b' 'e' 'm' 't']
Copying file:///tmp/tmpxbozysp3 [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/6.3 MiB.                                      
Creating Model


INFO:google.cloud.aiplatform.models:Creating Model


Create Model backing LRO: projects/412996116194/locations/us-central1/models/3154169006601011200/operations/6071307839106711552


INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/412996116194/locations/us-central1/models/3154169006601011200/operations/6071307839106711552


Model created. Resource name: projects/412996116194/locations/us-central1/models/3154169006601011200@1


INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/412996116194/locations/us-central1/models/3154169006601011200@1


To use this Model in another session:


INFO:google.cloud.aiplatform.models:To use this Model in another session:


model = aiplatform.Model('projects/412996116194/locations/us-central1/models/3154169006601011200@1')


INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/412996116194/locations/us-central1/models/3154169006601011200@1')


### Stop Experiment run

In [50]:
run.end_run()

### Visualize Experiment Lineage

Below you get the link to Vertex AI Metadata UI in the console that show the experiment lineage.

In [51]:
print("Open the following link:", exc.get_output_artifacts()[0].lineage_console_uri)

Open the following link: https://console.cloud.google.com/vertex-ai/locations/us-central1/metadata-stores/default/artifacts/f56097dd-a79a-4c8f-800f-9db3b7602a3e?project=sv-ml-sandbox


## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.

In [ ]:
# Delete experiment
exp = vertex_ai.Experiment(EXPERIMENT_NAME)
exp.delete()

# Delete model
model_list = vertex_ai.Model.list(filter=f'display_name="{MODEL_NAME}"')
for model in model_list:
    model.delete()

# Delete dataset
for dataset_name in [DATASET_NAME, PREPROCESSED_DATASET_NAME]:
    dataset_list = vertex_ai.TabularDataset.list(
        filter=f'display_name="{dataset_name}"'
    )
    for dataset in dataset_list:
        dataset.delete()

# Delete Cloud Storage objects that were created
delete_bucket = True

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI

!rm -Rf {DATA_PATH}

In [ ]:
!rm -Rf {DATA_PATH}